In [ ]:
"""
mlops/model.py - MLflow pyfunc Model Wrapper.

This module defines the `_PyfuncChatbot` class, a custom MLflow pyfunc model
wrapper. This class enables the entire LLM service to be packaged, logged,
and served using MLflow's standard model registry and serving capabilities.
"""
import json
import asyncio
import logging
import pandas as pd
from typing import Any, Dict, List, Optional, Union

import mlflow.pyfunc

# Set up logging for this module
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Import core components to be packaged with the model
from core.service import LLMService
from core.rag import RAGPipeline
from core.router import HeuristicRouter
from core.adapter import ControlHintAdapter
from core.guardrail import Guardrail

class _PyfuncChatbot(mlflow.pyfunc.PythonModel):
    """
    A custom MLflow pyfunc model wrapper for the LLMService.

    This class encapsulates the entire chatbot logic, including RAG, routing,
    and guardrails, into a single, portable artifact that can be deployed
    via MLflow's serving infrastructure.
    """
    def load_context(self, context: mlflow.pyfunc.model.PythonModelContext):
        """
        Loads the chatbot service and its components from the model artifacts.

        This method is called once when the model is loaded for serving. It
        initializes the RAG pipeline, adapter, router, and guardrail using
        the artifacts stored during model registration.
        """
        logger.info("Loading model context...")

        # Load artifacts: FAISS documents and the initial adapter hint
        docs_path = context.artifacts["docs"]
        hint_path = context.artifacts["hint"]
        with open(docs_path, "r", encoding="utf-8") as f:
            docs = json.load(f)
        with open(hint_path, "r", encoding="utf-8") as f:
            hint = f.read()

        # Initialize core components
        rag_pipeline = RAGPipeline(docs)
        adapter = ControlHintAdapter(hint)
        router = HeuristicRouter()
        guardrail = Guardrail()

        # Initialize the main LLMService
        self.service = LLMService(
            rag=rag_pipeline,
            adapter=adapter,
            router=router,
            guardrail=guardrail
        )
        # Create a new event loop for async operations in the predict method
        self.loop = asyncio.new_event_loop()

        logger.info("Model context loaded successfully.")

    def predict(self, context: mlflow.pyfunc.model.PythonModelContext,
                model_input: Union[pd.DataFrame, Dict[str, str]]) -> List[str]:
        """
        Generates a chatbot response based on the input question.

        This method is the main entry point for inference. It takes a DataFrame
        or dictionary as input, extracts the questions, and uses the `LLMService`
        to generate responses.

        Args:
            context: The MLflow context.
            model_input: A pandas DataFrame or dictionary with a 'question' column.

        Returns:
            A list of answer strings corresponding to each question.
        """
        logger.info("Starting prediction...")

        if isinstance(model_input, dict):
            # Allow for a single dictionary input
            questions = [model_input.get("question", "")]
        elif isinstance(model_input, pd.DataFrame):
            # Handle standard pandas DataFrame input
            questions = model_input["question"].tolist()
        else:
            raise TypeError("Model input must be a pandas DataFrame or a dictionary.")

        answers: List[str] = []
        for q in questions:
            # Run the async `answer` method from the LLMService
            ans = self.loop.run_until_complete(self.service.answer(q))
            answers.append(ans)

        logger.info("Prediction complete.")
        return answers